In [ ]:
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt

def calculate_nbr_score(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Assuming the CSV has a column 'sentiment' with values 'Positive', 'Negative', 'Neutral'
    sentiment_counts = df['sentiment'].value_counts()
    
    # Get counts for each sentiment
    positive_count = sentiment_counts.get('Positive', 0)
    negative_count = sentiment_counts.get('Negative', 0)
    neutral_count = sentiment_counts.get('Neutral', 0)
    
    # Total number of tweets
    total_count = positive_count + negative_count + neutral_count
    
    # Calculate NBR score
    nbr_score = (positive_count - negative_count) * 100 / total_count if total_count != 0 else 0
    
    return positive_count, negative_count, neutral_count, total_count, nbr_score

# File paths for the CSV files
files = {
    'Blinkit': 'blinkit_processed_tweets.csv',
    'Bigbasket': 'bigbasket_processed_tweets.csv',
    'Zepto': 'zepto_processed_tweets.csv',
    'Swiggy Instamart': 'swiggyinstamart_processed_tweets.csv'
}

# Dictionary to store the results
results = {}

# Calculate NBR score for each brand
for brand, file_path in files.items():
    pos, neg, neu, total, nbr = calculate_nbr_score(file_path)
    results[brand] = {
        'positive': pos,
        'negative': neg,
        'neutral': neu,
        'total': total,
        'nbr_score': nbr
    }

# Convert results to DataFrame for plotting
df_results = pd.DataFrame(results).T.reset_index()
df_results = df_results.rename(columns={'index': 'brand'})

# Plotting
# Pie charts for sentiment counts
sentiment_plot = (
    ggplot(df_results, aes(x='factor(1)', y='value', fill='variable')) +
    geom_bar(stat='identity', width=0.8) +
    facet_wrap('~ brand') +
    coord_polar(theta='y') +
    theme_void() +
    theme(legend_position='none') +
    scale_fill_manual(values=['forestgreen', 'deepskyblue', 'orangered']) +
    labs(y='Count', title='Sentiment Distribution by Brand')
)

# Bar plot for NBR scores
nbr_score_plot = (
    ggplot(df_results, aes(x='brand', y='nbr_score', fill='brand')) +
    geom_bar(stat='identity') +
    theme_bw() +
    theme(axis_text_x=element_text(rotation=45, hjust=1)) +
    labs(x='Brand', y='NBR Score', title='NBR Scores by Brand')
)

# Combine plots
plots = sentiment_plot | nbr_score_plot

# Display plots
print(plots)

# Save plots to file
ggsave(plots, "sentiment_nbr_plots.png", width=12, height=8, dpi=100)
